# ひろしまQuest2020 Predict

In [1]:
# 自分のマイドライブにマウントする(Google Colab使用時)

#from google.colab import drive
#drive.mount('/content/drive/')

In [2]:
#cd "/content/drive/MyDrive"

In [3]:
import numpy as np
import os
import pandas as pd

In [4]:
import tensorflow as tf
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
import pathlib
import shutil

### 推定

In [6]:
#TrainNotebookのモデルの保存先を指定
#model_pred = tf.keras.models.load_model('/content/drive/MyDrive/checkpoint_best')
#model_pred = tf.keras.models.load_model('./saved_model')
model_pred = tf.keras.models.load_model('./checkpoint_best')

# アーキテクチャを確認
model_pred.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 10, 10, 1024)      7037504   
_________________________________________________________________
dropout (Dropout)            (None, 10, 10, 1024)      0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1049600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 4100      
Total params: 8,091,204
Trainable params: 8,007,556
Non-trainable params: 83,648
_________________________________________

### サブミットファイルの作成

In [7]:
#dataset_path = pathlib.Path('/content/drive/MyDrive')#Google Colab使用時
dataset_path = pathlib.Path('.')
##os.listdir(dataset_path)

Submitファイルの準備

In [8]:
sample_df = pd.read_csv(dataset_path/'sample_submit.csv', header=None)
#sample_df.head()

#### テストデータの準備

In [9]:
test_folder_path = './test_images'
#test_folder_path = '/content/drive/MyDrive/test_images'

if not os.path.exists(test_folder_path):
    os.mkdir(test_folder_path) 
    
#os.listdir(test_folder_path)

In [10]:
test_dir = './test_images/all_classes'

In [11]:
#1回実行したらコメントアウトでOK（ドライブにファイルを作るのが目的のため）

if not os.path.exists(test_dir):
    shutil.copytree(test_folder_path, test_dir)


In [12]:
_, _, files = next(os.walk(test_dir))
file_count = len(files)
print("Number of pictures: ", file_count)

Number of pictures:  1651


In [13]:
test_image_generator = ImageDataGenerator(rescale=1./255)

In [14]:
IMG_HEIGHT = 320
IMG_WIDTH = 320

In [15]:
test_generator = test_image_generator.flow_from_directory(
    directory=test_folder_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode=None,
    shuffle=False
    )

Found 1651 images belonging to 1 classes.


## 推定

In [16]:
y_predict = model_pred.predict_generator(test_generator, verbose=1)

/Users/ishiitomoaki/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


52/52 [==============================] - 444s 9s/step


In [17]:
predicted_class_indices = np.argmax(y_predict, axis=1)
print("predicted class indices: ", predicted_class_indices)

#labels_dict = ({'優良': 0, '良': 1, '加工品': 2, '規格外': 3})
#labels = dict((v,k) for k,v in labels_dict.items())
#predictions = [labels[k] for k in predicted_class_indices]
#print("predicted class: ", predictions)

predicted class indices:  [0 3 0 ... 1 0 0]


### Submission

In [18]:
submit_ID = sample_df.iloc[:][0]
submit_TARGET = pd.DataFrame(predicted_class_indices)

submission_df = pd.concat([submit_ID, submit_TARGET], axis=1)
submit_columns = sample_df.columns
submission_df.columns = submit_columns

submission_df

,0,1
0,test_0000.jpg,0
1,test_0001.jpg,3
2,test_0002.jpg,0
3,test_0003.jpg,3
4,test_0004.jpg,0
...,...,...
1646,test_1646.jpg,2
1647,test_1647.jpg,2
1648,test_1648.jpg,1
1649,test_1649.jpg,0


In [19]:
submission_df.to_csv('submission.csv',index=False, header=False)

以上